In [11]:
import pandas as pd
import glob

In [56]:


def process_and_concat_sheets(file_path,monthname):
    # Load the Excel file
    excel_file = pd.ExcelFile(file_path)
    
    # List to store processed DataFrames
    all_data = []
    
    print('Excel sheets found:', excel_file.sheet_names)
    
    # Loop through all sheet names
    for sheet_name in excel_file.sheet_names:
        print('Processing sheet:', sheet_name)
        
        try:
            # Read the sheet
            file = pd.read_excel(file_path, sheet_name=sheet_name)

            # Process the data based on sheet name
            if "Mpay" in sheet_name:
                file['Loan No.'] = file.iloc[:, 10]  # 11th column (index 10)
                file['Payment'] = file.iloc[:, 5]   # 6th column (index 5)
                file['Date'] = file.iloc[:, 3]      # 4th column (index 3)
            else:
                file['Loan No.'] = file.iloc[:, 7]  # 8th column (index 7)
                file['Payment'] = file.iloc[:, 15]  # 16th column (index 15)
                file['Date'] = file.iloc[:, 4]      # 5th column (index 4)
                file['Month'] = monthname

            # Convert data types
            file['Loan No.'] = file['Loan No.'].astype(str)
            file['Date'] = file['Date'].astype(str)

            # Add 'Type' column
            file['Type'] = sheet_name

            # Keep only relevant columns
            file = file[['Loan No.', 'Payment', 'Type', 'Date','Month']]

            # Append to list
            all_data.append(file)
            print(f'Successfully processed: {sheet_name}')

        except Exception as e:
            print(f"Skipping sheet {sheet_name} due to error: {e}")
    
    # Concatenate all DataFrames if there are any valid ones
    if all_data:
        concatenated_data = pd.concat(all_data, ignore_index=True)
    else:
        concatenated_data = pd.DataFrame(columns=['Loan No.', 'Payment', 'Type', 'Date','Month'])

    return concatenated_data


In [57]:

file_list = glob.glob("./file/Payment_Oct-Dec2022_16-31/*")
# file_list = glob.glob(f"./file/*")


# List to store concatenated data from each file
all_files_data = []

# Loop through each file and process it
for file_path in file_list:
    print(f"Processing file: {file_path}")
    
    # Process the sheets in each file and concatenate the results
    file_data = process_and_concat_sheets(file_path,'Dec 22')
    
    # Append the data to the list
    all_files_data.append(file_data)

# Concatenate all file data into a single DataFrame
final_data = pd.concat(all_files_data, ignore_index=True)
final_data =final_data.dropna(subset=['Payment'])
final_data = final_data.loc[final_data['Loan No.'] != 'nan']
# final_data['Date'] = final_data['Date'].astype(str).str.replace(r'\.0$', '', regex=True).reset_index(drop=True)
print(final_data)

Processing file: ./file/Payment_Oct-Dec2022_16-31\ABC 16-12-2022.xlsm
Excel sheets found: ['BBL', 'CS', 'TESCO']
Processing sheet: BBL
Successfully processed: BBL
Processing sheet: CS
Successfully processed: CS
Processing sheet: TESCO
Successfully processed: TESCO
Processing file: ./file/Payment_Oct-Dec2022_16-31\ABC 17-12-2022.xlsm
Excel sheets found: ['BBL', 'CS', 'TESCO']
Processing sheet: BBL
Successfully processed: BBL
Processing sheet: CS
Successfully processed: CS
Processing sheet: TESCO
Successfully processed: TESCO
Processing file: ./file/Payment_Oct-Dec2022_16-31\ABC 18-12-2022.xlsm
Excel sheets found: ['BBL', 'CS', 'TESCO']
Processing sheet: BBL
Successfully processed: BBL
Processing sheet: CS
Successfully processed: CS
Processing sheet: TESCO
Successfully processed: TESCO
Processing file: ./file/Payment_Oct-Dec2022_16-31\ABC 19-12-2022.xlsm
Excel sheets found: ['BBL', 'TESCO', 'CS']
Processing sheet: BBL
Successfully processed: BBL
Processing sheet: TESCO
Successfully proce

In [58]:
final_data

,Loan No.,Payment,Type,Date,Month
0,5002764031002251.0,8300.0,BBL,15122022.0,Dec 22
1,9000220102001585.0,11429.0,BBL,15122022.0,Dec 22
2,5000161031046018.0,300.0,BBL,15122022.0,Dec 22
3,5000164101000154.0,200.0,BBL,15122022.0,Dec 22
4,7220550100000134.0,1000.0,BBL,15122022.0,Dec 22
...,...,...,...,...,...
46424,5000361021005988.0,900.0,TESCO,31122022.0,Dec 22
46425,5000159081090166.0,500.0,TESCO,31122022.0,Dec 22
46426,7220830100001859.0,500.0,TESCO,31122022.0,Dec 22
46427,5000460081093733.0,500.0,TESCO,31122022.0,Dec 22


In [59]:
final_data.to_excel('./output/Payment_Oct-Dec2022_16-31.xlsx')


In [9]:
def split_and_save_data(data, output_file):
    # Split data into chunks of 500,000 rows if needed
    chunk_size = 500000
    num_chunks = (len(data) // chunk_size) + (1 if len(data) % chunk_size > 0 else 0)
    
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        # Write each chunk to a separate sheet
        for chunk_num in range(num_chunks):
            start_row = chunk_num * chunk_size
            end_row = min((chunk_num + 1) * chunk_size, len(data))
            chunk = data.iloc[start_row:end_row]
            
            # Save the chunk to a new sheet
            chunk.to_excel(writer, sheet_name=f'Sheet_{chunk_num + 1}', index=False)
            print(f"Saved Sheet_{chunk_num + 1} with rows {start_row} to {end_row}")

In [21]:
output_file = './output/0225.xlsx'
if len(final_data) > 500000:
        print(f"Total rows {len(final_data)} exceed 500,000. Splitting into multiple sheets.")
        split_and_save_data(final_data, output_file)
else:
    # If it's less than 500,000 rows, save it as one sheet
    final_data.to_excel(output_file, index=False, sheet_name='Sheet_1')
    print(f"Saved all data to {output_file}.")

Total rows 1297134 exceed 500,000. Splitting into multiple sheets.
Saved Sheet_1 with rows 0 to 500000
Saved Sheet_2 with rows 500000 to 1000000
Saved Sheet_3 with rows 1000000 to 1297134
